In [ ]:
# 실사 이미지 네이밍 규칙 변경 v1.0

import natsort # 숫자로 된 파일명 정렬 모듈
import numpy as np
import os
import pandas as pd
import shutil # 파일, 폴더 복사 모듈

df = pd.read_excel('C:\\Users\\user\\Desktop\\국토부과제_2차년도_엑스레이 이미지 데이터 DB_ver0.10.xlsx', header=None)

DIR_WORKING = '\\\\{NAS 주소}\\2_팀 공유 폴더\\1) X-Ray 촬영 이미지 & 관리 문서\\2) 3D X-Ray 이미지_SSTLabs\\SSTLabs 3D 촬영 이미지\\20220412~20220413 촬영데이터\\개봉 실사 이미지_신버전\\Round 0 확인'
DIR_WORKING_RESULT = DIR_WORKING + '/Result/' + DIR_WORKING.split('\\')[-1]

if not os.path.exists(DIR_WORKING_RESULT):
    shutil.copytree(DIR_WORKING, DIR_WORKING_RESULT) # 원본 폴더(dir_working)는 보관용으로 남기고, copy 폴더를 생성하여 작업.
    
row = 1 # 엑셀 시트의 시작 행 번호
skip = 0 # 고정

filelist = os.listdir(DIR_WORKING_RESULT) # 해당 경로에 있는 디렉토리, 파일 가져오기
filelist = natsort.natsorted(filelist) # 파일명이 숫자로 되어 있어, 정렬

for file in filelist:
    os.chdir(DIR_WORKING_RESULT) # change directory
    
    newname = df.values[row + skip*9][10][:-2] + ".jpg" # 개봉 실사 이미지 파일명 : 촬영 각도(맨 뒷자리)를 제외한 엑스레이 이미지 파일명
    os.rename(DIR_WORKING_RESULT + "/" + file, DIR_WORKING_RESULT + "/" + newname) # 파일명 변경

    # 체크 리스트
    print(file) # 변경할 개봉 실사 이미지 파일명(구) -> ex. 517
    print(newname) # 
    print("-------------")

    skip = skip + 1

print(row + skip * 9)

In [ ]:
# 엑스레이 이미지 파일 변환ver0.1

import cv2
import natsort # 숫자로 된 파일명 정렬 모듈
import numpy as np
import os
import pandas as pd
import shutil # 파일, 폴더 복사 모듈

df = pd.read_excel('C:\\Users\\user\\Desktop\\[NIA] Runlist_1cycle_E3S690_Rev1_Draft.xlsx', header=None) # 엑셀 파일 위치 지정

DIR_WORKING = 'C:\\Users\\user\\Desktop\\X-ray Data' # 엑스레이 이미지 Raw 데이터가 모여져 있는 폴더 경로
DIR_WORKING_RESULT = DIR_WORKING + '\\Result' # 변환 후 이미지 저장 폴더 경로

# 변환할 이미지 크기 설정
HEIGHT = 760
WIDTH = 896

row = 1 # 엑셀 시트의 시작 행 번호

if not os.path.exists(DIR_WORKING_RESULT):
    shutil.copytree(DIR_WORKING, DIR_WORKING_RESULT) # 원본 폴더(dir_working)는 보관용으로 남기고, Result 폴더를 생성하여 작업.

filelist = os.listdir(DIR_WORKING_RESULT) # Result 폴더 안에 있는 디렉토리 및 파일 읽기

for file in filelist:
    os.chdir(DIR_WORKING_RESULT) # change directory

    # read image as binary
    read_binary = open(DIR_WORKING_RESULT + '/' + file, "rb") # 1. 파일 경로, 2. read + binary (rb)
    img = np.fromfile(read_binary, dtype = 'uint16', sep = "", count = (HEIGHT*WIDTH), offset = 0)
    img = np.reshape(img, [HEIGHT,WIDTH]) # 760 -> height, 896 -> width
    
    markfile = df.values[row][10] + '.jpg'
    
    cv2.imwrite(markfile, img)
    os.chdir(DIR_WORKING_RESULT)

    # 체크 리스트
    print(file)
    print(df.values[row][2])
    print(markfile)

    if os.path.exists(DIR_WORKING_RESULT + '/' + file):
        os.remove(DIR_WORKING_RESULT +'/'+ file) # 이미지 변환이 끝난 raw 파일은 필요 없으므로 삭제함.
    
    read_binary.close()
    
    row = row + 1
        
print(row)


In [ ]:
# 원천 데이터 - superb AI S3 업로드 폴더 구조로 변경

import shutil # 파일, 폴더 복사 모듈
import os
import numpy as np
import pandas as pd
import natsort # 숫자로 된 파일명 정렬 모듈

df = pd.read_excel('C:/Users/user/Desktop/[NIA] 1cycle_엑스레이 이미지 파일명.xlsx', header=None) # 엑셀 파일 위치 지정

DIR_DESKTOP = "C:/Users/user/Desktop"
DIR_WORKING_PHOTO = DIR_DESKTOP + "/혼재(360회)" # 촬영 데이터가 모여져 있는 폴더 경로
DIR_WORKING_XRAY = DIR_DESKTOP + "/color"
DIR_WORKING_XRAY_COPY = DIR_DESKTOP + "/copy"
DIR_WORKING_RESULT_BASE = DIR_DESKTOP + '/NIA'

if not os.path.exists(DIR_WORKING_XRAY_COPY):
    shutil.copytree(DIR_WORKING_XRAY, DIR_WORKING_XRAY_COPY) 
if not os.path.exists(DIR_WORKING_RESULT_BASE):
    shutil.copytree(DIR_WORKING_PHOTO, DIR_WORKING_RESULT_BASE) 
if not os.path.exists(DIR_WORKING_RESULT_BASE):
    os.mkdir(DIR_WORKING_RESULT_BASE)

row = 2161 # 파일 명 찾는 가변 변수
skip = 0

def photo(row, skip):
    filelist = os.listdir(DIR_WORKING_PHOTO)

    filelist = natsort.natsorted(filelist)

    for file in filelist:

        folder = df.values[row + skip*9][10][:-2]
        newname = folder + "_0.jpg"

        if not os.path.exists(DIR_WORKING_RESULT_BASE + "/" + folder):
            os.mkdir(DIR_WORKING_RESULT_BASE + "/" + folder)

        os.chdir(DIR_WORKING_RESULT_BASE + "/" + folder) # change directory
        os.rename(DIR_WORKING_RESULT_BASE + "/" + file, DIR_WORKING_RESULT_BASE + "/" + folder + "/" + newname)

        # 확인용
        print("file : %s, Excel : %s" %(file, df.values[row + skip*9][13]))
        print("folder : %s" %folder)
        print("filename : %s" %newname)
        print("-------------")
        
        skip = skip + 1
    return row + skip * 9

def xray(row):
    for (root, dirs, files) in os.walk(DIR_WORKING_XRAY_COPY):
        for file in files:
            if("he" == file.split("_")[1][:2]): # 'he' 붙어 있는 이미지만 사용
                folder = df.values[row][10][:-2]
                newname = df.values[row][10] + ".jpg"

                if not os.path.exists(DIR_WORKING_RESULT_BASE + "/" + folder):
                    os.mkdir(DIR_WORKING_RESULT_BASE + "/" + folder)

                os.chdir(DIR_WORKING_RESULT_BASE + "/" + folder) # change directory
                os.rename(root + "/" + file, DIR_WORKING_RESULT_BASE + "/" + folder + "/" + newname)
                
                print("check : %s" %df.values[row][2])
                print("folder : %s, file : %s" %(root.split("\\")[1], file))
                print("newname : %s" %newname)
                print("--------------------------------")

                if os.path.exists(root + "/" + file): # 이미지 변환이 끝난 raw 파일은 필요 없으므로 삭제
                    os.remove(root + "/" + file)

                row = row + 1

xray(row)
print("\n\n\n----------------\n여기서부터 실사 이미지\n-----------\n\n\n")
# photo(row, skip)

In [ ]:
# [번외] 엑스레이 이미지 파일 변환 ver0.2 : 하단 이미지가 잘리는 현상 보완 + 라벨링 데이터 폴더 구조

import cv2 # openCV 모듈 
import numpy as np
import os
import pandas as pd
import shutil # 파일, 폴더 복사 모듈

df = pd.read_excel('C:/Users/user/Desktop/[NIA] Runlist_1cycle_E3S690_Rev1_Draft.xlsx', header=None) # 엑셀 파일 위치 지정

# 디렉토리 설정
DIR_DESKTOP = "C:/Users/user/Desktop"
DIR_WORKING = DIR_DESKTOP + "/X-ray Data" # Raw 엑스레이 이미지가 모여져 있는 폴더 경로
DIR_WORKING_COPY = DIR_WORKING + '/temp' # Raw -> JPG 엑스레이 이미지 전체 크기 저장 폴더
DIR_RESULT = DIR_DESKTOP + '/Result'# 최종 사이즈 (896, 760)으로 자른 JPG 엑스레이 이미지 저장 폴더

# 변환할 이미지 크기 설정
HEIGHT = 760
WIDTH = 896

row = 1 # 엑셀 시트의 시작 행 번호

if not os.path.exists(DIR_WORKING_COPY):
    shutil.copytree(DIR_WORKING, DIR_WORKING_COPY) # 원본 폴더(dir_working)는 보관용으로 남기고, copy 폴더를 생성하여 작업.
if not os.path.exists(DIR_RESULT):
    os.mkdir(DIR_RESULT)

filelist = os.listdir(DIR_WORKING_COPY)

for file in filelist:

    # raw 이미지 -> png 이미지로 전체 크기로 바꾸기 코드
    os.chdir(DIR_WORKING_COPY) # change directory
    
    read_binary = open(DIR_WORKING_COPY+'/'+file, "rb") # 1. 파일 경로, 2. read + binary (rb)
    img = np.fromfile(read_binary, dtype='uint16', sep="")
    img = np.reshape(img, [img.shape[0]//WIDTH, WIDTH]) # height -> 전체 count // width, 896 -> width
    read_binary.close()
    
    markfile = df.values[row][10] + '.png' # Result 폴더에 최종 저장될 이미지 파일 이름
    
    cv2.imwrite(markfile, img)

    # 파일 변환 확인용
    print(file)
    print(markfile)
    print(df.values[row][2])

    # 896, 760 크기로 변환 + 물품 분류 체계에 따라 폴더 나눠서 저장
    # 폴더 구조 : NIA(Root) - 식별목적 - 물품 Class - 물품 DB - 촬영 방식
    
    if 1 <= int(df[6][row][:3]) <= 200: # 식별목적(대분류)
        main_cat = "위해물품"
    elif 201 <= int(df[6][row][:3]) <= 300:
        main_cat = "정보저장매체"
    elif 301 <= int(df[6][row][:3]) <= 999:
        main_cat = "일반물품"
    else:
        main_cat = "error"

    object_class = df[5][row] # 물품 Class
    object_db = df[11][row] # 물품 DB

    if df[4][row] == "S": # 촬영 방식
        shot_type = "단일"
    elif df[4][row] == "P":
        shot_type = "복수"
    elif df[4][row] == "M":
        shot_type = "혼재"
    else:
        shot_type = "error"

    dir_main_cat = DIR_RESULT + "/" + main_cat
    dir_object_class = dir_main_cat + "/" + object_class
    dir_object_db = dir_object_class + "/" + object_db
    dir_shot_type = dir_object_db + "/" + shot_type

    if not os.path.exists(dir_main_cat):
        os.mkdir(dir_main_cat)
    if not os.path.exists(dir_object_class):
        os.mkdir(dir_object_class)
    if not os.path.exists(dir_object_db):
        os.mkdir(dir_object_db)
    if not os.path.exists(dir_shot_type):
        os.mkdir(dir_shot_type)

    # 이미지의 하단이 짤리는 현상 방지 코드
    img_detection = cv2.imread(markfile, cv2.IMREAD_COLOR) # image detection code
    gray = cv2.cvtColor(img_detection, cv2.COLOR_BGR2GRAY) # 이미지 그레이 변환
    blur = cv2.GaussianBlur(gray, ksize=(5,5), sigmaX=0)  # 가우시안 블러 사용 -> 이미지를 부드럽게 하기에 배경과 원하지 않는 부분을 제거하는 효과
    
    redlalwlt, th1 = cv2.threshold(blur, 150, 255, cv2.THRESH_BINARY_INV) # 임계값 설정
    # binary = cv2.bitwise_not(th1)

    contours, hierarchy = cv2.findContours(th1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # contours -> 3차원
    contours_xy = np.array(contours) # 윤곽선 최소 좌표를 구하기 위해 변형

    value = list()

    for x in range(len(contours_xy)): 
        for y in range(len(contours_xy[x])):
            value.append(contours_xy[x][y][0][1]) # height 값

    y_min = min(value) # height 시작 위치 조정
    
    if (y_min - 55 <= 0):
        y_min = 0
    else:
        y_min = y_min - 55
    
    img = img[y_min:y_min+HEIGHT, :]

    if os.path.exists(DIR_WORKING_COPY + '/' + file): # 이미지 변환이 끝난 raw 파일은 필요 없으므로 삭제
        os.remove(DIR_WORKING_COPY +'/'+ file)
    
    os.chdir(DIR_RESULT) # Result 폴더
    cv2.imwrite(markfile, img) # 이미지 저장
    
    # 가변 변수 1 증가  
    row = row + 1

# 마지막 레코드 -> i 시작 번호에 복사해서 붙여넣기
print(row)
